In [263]:
import pandas as pd
import numpy as np
import re
import gc

from datetime import datetime, timedelta

In [2]:
import psutil
psutil.virtual_memory()

svmem(total=17179869184, available=6245736448, percent=63.6, used=8584216576, free=698503168, active=5433950208, inactive=4199198720, wired=3150266368)

# Definitions

In [350]:
# Remove digits from location data so we have "City, State" format
def remove_digits(df):
    """Remove digits from location"""
    df["Location_Clean"] = df["Location"].replace(to_replace = r'\d', value='', regex=True).str.rstrip()
    
    return df


# Create a day index which will be used to calcultate when the job was posted
def day_index(df):
    """Create a day index that represents how long ago the job was posted """
    
    df_dummy = pd.DataFrame()
    df_dummy["Day Index"] = df["Date"].str.split(" ", expand=True).get(0)
    df["Day Index"] = df_dummy["Day Index"].apply(lambda x: 0 if (x == 'Just' or x == 'Today') else x)
    
    return df

# Clean location data for new week data

In [340]:
df_usa_april_week3 = pd.read_csv('df_usa_april_week3.csv')

### Create a locationations df to be mapped back into original df 
location = pd.DataFrame(df_usa_april_week3['Location'].str.split(">", expand=True).get(1).str.split("<", expand=True).get(0))
location = location.rename(columns={0: 'Location'})

### Runs the remove digits function from locationation column
location.pipe(remove_digits)

df_usa_april_week3["Location"] = location["Location_Clean"]

df_usa_april_week3.head()

,Title,Location,Company,Rating,Salary,Sponsored,Date,As of Date,Description,Industry
0,Data Scientist Entry Level - Pathrise Recruiti...,Remote,Pathrise,None,None,None,22 days ago,2020-04-26,What is Pathrise Recruiting Partners?\n\nPathr...,Datascience
1,Data Science Analyst,"Ann Arbor, MI",Domino's,3.6,None,None,30+ days ago,2020-04-26,"Domino’s Pizza, which began in 1960 as a singl...",Datascience
2,Junior Data Scientist,"New York, NY",1-800-Flowers,3.7,None,None,10 days ago,2020-04-26,As part of the NYC Innovation Team located in ...,Datascience
3,Data Scientist/Analytics Consultant,"Arlington, VA",Deloitte,4.0,None,None,12 days ago,2020-04-26,"Are you an analytical, data-driven professiona...",Datascience
4,Data Science & Analytics,"South Jordan, UT",Digital Trike,None,None,None,30+ days ago,2020-04-26,"As much as you’re a master at rigorous mining,...",Datascience


### Remove 30+ Days old jobs for the purpose of understanding trends in job postings counts

In [341]:
df_usa_filter_april_week3=df_usa_april_week3.copy()[df_usa_april_week3["Date"]!='30+ days ago']

print("Unique jobs in usa as of week 3: " + str(df_usa_april_week3.shape[0]))
print("Unique jobs in usa as of week 3 but not including old jobs: " + str(df_usa_filter_april_week3.shape[0]))


# Create a day index to be used in created "Date Posted"
df_usa_filter_april_week3.pipe(day_index).head()

# Format variables
df_usa_filter_april_week3["Day Index"]=df_usa_filter_april_week3["Day Index"].astype(float)
df_usa_filter_april_week3["As of Date"] = pd.to_datetime(df_usa_filter_april_week3["As of Date"])

# Create Date Posted variable representing the date on which the job was posted 
df_usa_filter_april_week3["Date Posted"] = df_usa_filter_april_week3["As of Date"] - pd.to_timedelta(df_usa_filter_april_week3["Day Index"], unit='d')

df_usa_filter_april_week3.head()

Unique jobs in usa as of week 3: 12938
Unique jobs in usa as of week 3 but not including old jobs: 5377


,Title,Location,Company,Rating,Salary,Sponsored,Date,As of Date,Description,Industry,Day Index,Date Posted
0,Data Scientist Entry Level - Pathrise Recruiti...,Remote,Pathrise,None,None,None,22 days ago,2020-04-26,What is Pathrise Recruiting Partners?\n\nPathr...,Datascience,22.0,2020-04-04
2,Junior Data Scientist,"New York, NY",1-800-Flowers,3.7,None,None,10 days ago,2020-04-26,As part of the NYC Innovation Team located in ...,Datascience,10.0,2020-04-16
3,Data Scientist/Analytics Consultant,"Arlington, VA",Deloitte,4.0,None,None,12 days ago,2020-04-26,"Are you an analytical, data-driven professiona...",Datascience,12.0,2020-04-14
5,Data Scientist Analyst,"Sunnyvale, CA",LOCKHEED MARTIN CORPORATION,4.0,None,None,2 days ago,2020-04-26,Perform exploratory research and analysis to i...,Datascience,2.0,2020-04-24
6,"Data Science Intern (Summer, 2020)","Westfield Center, OH",Westfield Insurance,4.0,None,None,1 day ago,2020-04-26,"BUILD YOUR FUTURE, WHILE PROTECTING THEIRS.\nY...",Datascience,1.0,2020-04-25


## Test if the number of postings counts by Date Posted and Location makes sense

In [348]:
counts_april = df_usa_filter_april_week3.groupby('Date Posted').count()["Title"]
counts_april.tail(10)

Date Posted
2020-04-22    305
2020-04-23    384
2020-04-24    419
2020-04-25    508
2020-04-26    165
2020-04-27    312
2020-04-28    180
2020-04-29    167
2020-04-30    237
2020-05-01    468
Name: Title, dtype: int64

In [347]:
df_usa_filter_april_week3.groupby("Location").count()["Title"].sort_values(ascending=False).head(10)

Location
New York, NY         140
Washington, DC        81
Seattle, WA           75
Chicago, IL           66
San Francisco, CA     64
Atlanta, GA           59
United States         55
Remote                48
Charlotte, NC         44
Dallas, TX            44
Name: Title, dtype: int64

In [349]:
df_usa_filter_april_week3_formerge=df_usa_filter_april_week3.copy()[(df_usa_filter_april_week3["Date Posted"] >= datetime(2020,4,19)) &
                                                    (df_usa_filter_april_week3["Date Posted"] <= datetime(2020,4,25))]

counts_april = df_usa_filter_april_week3_formerge.groupby('Date Posted').count()["Title"]
counts_april

Date Posted
2020-04-19     47
2020-04-20     23
2020-04-21    218
2020-04-22    305
2020-04-23    384
2020-04-24    419
2020-04-25    508
Name: Title, dtype: int64

# Import the dataset needed to be merged with the new week data

In [354]:
df_usa=pd.read_csv('df_usa_april_week2_merged_no30days.csv')

# Set the format of the variables
df_usa["Date Posted"]=pd.to_datetime(df_usa["Date Posted"])
df_usa["As of Date"]=pd.to_datetime(df_usa["As of Date"])
df_usa["Day Index"]=df_usa["Day Index"].astype(int)

df_usa.dtypes

Title                  object
Company                object
Rating                 object
Salary                 object
Sponsored              object
Date                   object
As of Date     datetime64[ns]
Description            object
Industry               object
Location               object
Day Index               int64
Date Posted    datetime64[ns]
dtype: object

In [359]:
# Merge the two datasets

df_usa_total=pd.concat([df_usa_filter_april_week3_formerge,df_usa], axis=0, ignore_index=True, sort = False)
print("Total number of job postings: " +   str(df_usa_total.shape[0]))

df_usa=df_usa_total.drop_duplicates()
print("Total number of unique job postings: " + str(df_usa.shape[0]))

Total number of job postings: 14203
Total number of unique job postings: 12299


In [361]:
# Test if the numbed of job counts by Date Posted make sense

df_usa.groupby("Date Posted").count()["Title"]

Date Posted
2020-03-04    122
2020-03-05    119
2020-03-06    192
2020-03-07    180
2020-03-08     61
2020-03-09     27
2020-03-10    191
2020-03-11    246
2020-03-12    251
2020-03-13    211
2020-03-14    180
2020-03-15     35
2020-03-16     31
2020-03-17    132
2020-03-18    208
2020-03-19    234
2020-03-20    228
2020-03-21    245
2020-03-22     69
2020-03-23     32
2020-03-24    246
2020-03-25    304
2020-03-26    272
2020-03-27    317
2020-03-28    332
2020-03-29    104
2020-03-30     44
2020-03-31    387
2020-04-01    508
2020-04-02    615
2020-04-03    394
2020-04-04    416
2020-04-05     72
2020-04-06     38
2020-04-07    232
2020-04-08    315
2020-04-09    343
2020-04-10    380
2020-04-11    364
2020-04-12     62
2020-04-13     27
2020-04-14    203
2020-04-15    331
2020-04-16    312
2020-04-17    300
2020-04-18    484
2020-04-19     47
2020-04-20     23
2020-04-21    217
2020-04-22    305
2020-04-23    384
2020-04-24    419
2020-04-25    508
Name: Title, dtype: int64

## Export the merged dataset

In [363]:
df_usa.to_csv('df_usa_april_week3_merged_no30days.csv')